# VacationPy

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from config import g_key
from config import weather_api_key
from citipy import citipy
from pprint import pprint

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_weather_data = pd.read_csv("../WeatherPy/Output_Data/Clean City Data.csv")
city_weather_df = pd.DataFrame(city_weather_data)
city_weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hemisphere
0,Esperance,-33.87,121.90,53.60,100,40,8.05,AU,1601419994,Southern Hemisphere
1,Ushuaia,-54.80,-68.30,35.60,80,75,26.40,AR,1601420042,Southern Hemisphere
2,East London,-33.02,27.91,59.29,67,97,1.81,ZA,1601419984,Southern Hemisphere
3,Hithadhoo,-0.60,73.08,82.99,74,80,8.84,MV,1601420043,Southern Hemisphere
4,Khatanga,71.97,102.50,31.10,99,100,10.13,RU,1601420043,Northern Hemisphere


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# set locations and humidity values for heatmap
locations = city_weather_df[["Lat", "Lng"]]

humidity_list = city_weather_df['Humidity'].to_list()
# Find max humidity value for to set the intensity factor in the heatmap layer
h_intensity = (max(humidity_list))

gmaps.configure(g_key)

fig = gmaps.figure(layout={
        'width': '800px',
        'height': '500px',
        'padding': '3px',
        'border': '1px solid black',
        'map_type': "HYBRID"
})

fig.add_layer(gmaps.heatmap_layer(locations, weights=humidity_list,max_intensity=h_intensity, ))


fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', padding='3px', width='800px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows with null values.

In [4]:
# Ideal cities criteria
# Temp: Between 80 and 95
# Humidity: Less Than 65
# Cloudiness: Less than 30%
# Wind Speed: Between 5 and 15

ideal_cities_df = city_weather_df
ideal_cities_df = ideal_cities_df.loc[((ideal_cities_df["Max Temp"] < 95) & (ideal_cities_df["Max Temp"] > 79)) & \
                                      (ideal_cities_df["Humidity"] <= 65) & (ideal_cities_df["Cloudiness"] <= 30) & \
                                      ((ideal_cities_df["Wind Speed"] <= 15) & (ideal_cities_df["Wind Speed"] >= 5))]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna(axis=0, how="any")
ideal_cities_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hemisphere
24,Kapaa,22.08,-159.32,86.0,62,1,6.93,US,1601419830,Northern Hemisphere
26,Jalu,29.03,21.55,83.3,36,0,10.94,LY,1601420050,Northern Hemisphere
33,Boa Vista,2.82,-60.67,89.6,46,20,5.82,BR,1601420051,Northern Hemisphere
169,Esmeraldas,-19.76,-44.31,87.8,33,20,8.05,BR,1601420090,Southern Hemisphere
179,Cabo San Lucas,22.89,-109.91,91.4,58,5,14.99,MX,1601420093,Northern Hemisphere


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels within 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
hotel_df = ideal_cities_df
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hemisphere,Hotel Name
24,Kapaa,22.08,-159.32,86.0,62,1,6.93,US,1601419830,Northern Hemisphere,
26,Jalu,29.03,21.55,83.3,36,0,10.94,LY,1601420050,Northern Hemisphere,
33,Boa Vista,2.82,-60.67,89.6,46,20,5.82,BR,1601420051,Northern Hemisphere,
169,Esmeraldas,-19.76,-44.31,87.8,33,20,8.05,BR,1601420090,Southern Hemisphere,
179,Cabo San Lucas,22.89,-109.91,91.4,58,5,14.99,MX,1601420093,Northern Hemisphere,


In [6]:
hotel_list = []


distance = '5000'
act_type = 'Hotel'
keyword = 'Hotel'

#API structure from API documentation
#"https://maps.googleapis.com/maps/api/place/nearbysearch/json?
#location=-33.8670522,151.1957362&radius=1500&type=restaurant&keyword=cruise&key=YOUR_API_KEY"

hotel_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location='
    
for index, row in hotel_df.iterrows():
    latitude = row["Lat"]
    longitude = row["Lng"]

    hotel_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=" 
    location = f'{latitude},{longitude}'
    hotels = (hotel_url + location + '&radius=' + distance + '&type=' + act_type + '&keyword=' + keyword + '&key=' + g_key)

    response = requests.get(hotels)

    response = response.json()

    try:
        hotel_list.append(response['results'][0]['name'])        
    except:
        hotel_list.append(np.nan)  

hotel_df["Hotel Name"] = hotel_list

In [7]:
# Drop any records that do not include a hotel name
hotel_df.dropna(axis=0, how="any", inplace=True)

In [8]:
# Limit the results to 10 locations
hotel_df = hotel_df.head(10)
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hemisphere,Hotel Name
24,Kapaa,22.08,-159.32,86.0,62,1,6.93,US,1601419830,Northern Hemisphere,Hilton Garden Inn Kauai Wailua Bay
26,Jalu,29.03,21.55,83.3,36,0,10.94,LY,1601420050,Northern Hemisphere,Jalu Hotel
33,Boa Vista,2.82,-60.67,89.6,46,20,5.82,BR,1601420051,Northern Hemisphere,Hotel Orla do Rio Branco
169,Esmeraldas,-19.76,-44.31,87.8,33,20,8.05,BR,1601420090,Southern Hemisphere,Parque do Avestruz Hotel Fazenda
179,Cabo San Lucas,22.89,-109.91,91.4,58,5,14.99,MX,1601420093,Northern Hemisphere,Welk Resorts Cabo San Lucas - Sirena del Mar


In [13]:
hotel_df.describe()

,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,24.00000,24.000000,24.000000,24.000000,24.000000,24.000000,2.400000e+01
mean,13.72000,-45.080833,85.998750,38.458333,5.500000,9.541667,1.601420e+09
std,21.98449,83.695290,4.503242,16.078555,8.166741,3.218800,1.060893e+02
min,-27.80000,-159.320000,79.390000,10.000000,0.000000,5.820000,1.601420e+09
25%,-1.44750,-110.427500,82.342500,25.750000,0.000000,6.930000,1.601420e+09
50%,22.73000,-59.735000,84.580000,36.500000,1.000000,8.230000,1.601420e+09
75%,28.49000,26.215000,90.032500,53.500000,7.250000,12.615000,1.601420e+09
max,41.12000,129.550000,94.640000,64.000000,20.000000,14.990000,1.601420e+09


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', padding='3px', width='800px'))